In [ ]:
import time
import json
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

BASE_URL = "https://dic.hankyung.com/economy/list/?phoneme={phoneme}"

DOMAIN = "Marketing / Economy"
DEPT_ID = "002"

# 드라이버 생성
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 필요시 비활성화 가능
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=options)
    return driver


# 용어 더보기 전부 클릭
def click_all_more_terms(driver):
    while True:
        try:
            btn = driver.find_element(By.ID, "moreBtn")

            if not btn.is_displayed() or not btn.is_enabled():
                print("더보기 버튼 비활성 → 종료")
                break

            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1.0)

        except NoSuchElementException:
            print("더보기 버튼 없음 → 종료")
            break
        except ElementClickInterceptedException:
            print("더보기 버튼 클릭 가로막힘 → 재시도")
            time.sleep(1)
            try:
                btn = driver.find_element(By.ID, "moreBtn")
                driver.execute_script("arguments[0].click();", btn)
            except:
                break
        except Exception as e:
            print("예외 발생, 종료:", e)
            break

# HTML → 용어 파싱
def parse_terms_from_page(html):
    soup = BeautifulSoup(html, "html.parser")
    results = []

    for li in soup.select("div.list_sty ul li"):
        a_tag = li.select_one("h4 a")
        p_tag = li.select_one("p")

        if not a_tag or not p_tag:
            continue

        title = a_tag.get_text(strip=True)
        answer = p_tag.get_text(" ", strip=True)

        results.append({
            "title": title,
            "answer": answer,
            "domain": DOMAIN,
            "dept_id": DEPT_ID
        })

    return results

# 단일 phoneme 크롤링
def crawl_one_phoneme(phoneme):
    print(f"\n===== [{phoneme}] 수집 시작 =====")

    driver = create_driver()
    url = BASE_URL.format(phoneme=phoneme)
    driver.get(url)
    time.sleep(1.5)

    click_all_more_terms(driver)

    html = driver.page_source
    terms = parse_terms_from_page(html)

    driver.quit()
    print(f"[{phoneme}] 수집 완료 → {len(terms)} 개")
    return terms


# 전체 루프
if __name__ == "__main__":

    phonemes = ["ㄱ","ㄴ","ㄷ","ㄹ","ㅁ","ㅂ","ㅅ","ㅇ","ㅈ","ㅊ","ㅋ","ㅌ","ㅍ","ㅎ","ABC"]

    all_terms = []

    for p in phonemes:
        data = crawl_one_phoneme(p)
        all_terms.extend(data)

        # 개별 저장
        with open(f"hankyung_{p}.json", "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    # 전체 통합본 저장
    with open("hankyung_all_terms.json", "w", encoding="utf-8") as f:
        json.dump(all_terms, f, ensure_ascii=False, indent=2)

    print("\n======================================")
    print("전체 크롤링 완료")
    print("총 용어 수:", len(all_terms))
    print("파일: hankyung_all_terms.json 생성됨")



===== [ㄱ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㄱ] 수집 완료 → 1744 개

===== [ㄴ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㄴ] 수집 완료 → 326 개

===== [ㄷ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㄷ] 수집 완료 → 574 개

===== [ㄹ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㄹ] 수집 완료 → 311 개

===== [ㅁ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅁ] 수집 완료 → 700 개

===== [ㅂ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅂ] 수집 완료 → 954 개

===== [ㅅ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅅ] 수집 완료 → 1614 개

===== [ㅇ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅇ] 수집 완료 → 1872 개

===== [ㅈ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅈ] 수집 완료 → 1399 개

===== [ㅊ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅊ] 수집 완료 → 437 개

===== [ㅋ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅋ] 수집 완료 → 357 개

===== [ㅌ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅌ] 수집 완료 → 457 개

===== [ㅍ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅍ] 수집 완료 → 619 개

===== [ㅎ] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ㅎ] 수집 완료 → 730 개

===== [ABC] 수집 시작 =====
더보기 버튼 비활성 → 종료
[ABC] 수집 완료 → 1150 개

전체 크롤링 완료
총 용어 수: 13244
파일: hankyung_all_terms.json 생성됨
